In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from skimage.transform import resize
import glob
from random import shuffle

%matplotlib inline

width = 64
height = 64

images = glob.glob('catsvsdogs/train/*.jpg')
labels = [0 if '\cat' in addr else 1 for addr in images]

def resizeImages(images):
    temp = []
    for img in images:            
        image = np.array(plt.imread(img))
        image = scipy.misc.imresize(image, size=(width,height))
        temp.append(image)
    return np.array(temp)

classes = ['cat', 'dog']
   
train_X = resizeImages(images)
train_Y = np.array((labels,))
m_train = train_X.shape[0]

In [ ]:
train_X = train_X.reshape(train_X.shape[0], -1).T/255.

In [ ]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [ ]:
def initialize_with_zeros(dim):
    w = np.zeros((dim,1))
    b = 0
    return w, b

In [ ]:
def propagate(w, b, X, Y):
    m = X.shape[1]
    
    A = sigmoid(np.dot(w.T, X) + b)
    cost = (-1/m)*np.sum(Y*np.log(A) + (1-Y)*np.log(1-A))
    
    dw = (1/m)*np.dot(X, (A-Y).T)
    db = (1/m)*np.sum(A - Y)
    
    cost = np.squeeze(cost)
    grads = {"dw": dw, "db": db}
    
    return grads, cost

In [ ]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    costs = []
    
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y) 
        dw = grads["dw"]
        db = grads["db"] 
        w = w - learning_rate*dw
        b = b - learning_rate*db   
        if i % 100 == 0:
            costs.append(cost)
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
            
    params = {"w": w, "b": b}
    grads = {"dw": dw, "db": db}
    return params, grads, costs

In [ ]:
def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    A = sigmoid(np.dot(w.T, X) + b)
    
    for i in range(A.shape[1]):
        if A[0,i] >= 0.5:
            Y_prediction[0,i] = 1
        else:
            Y_prediction[0,i] = 0
            
    return Y_prediction

In [1]:
def model(X_train, Y_train, num_iterations = 500000, learning_rate = 0.003, print_cost = False):
    w, b = initialize_with_zeros(X_train.shape[0])
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    w = parameters["w"]
    b = parameters["b"]
    
    Y_prediction_train = predict(w, b, X_train)
    
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    
    d = {"costs": costs,
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [2]:
def predictImage(imagePath):
    image = np.array(ndimage.imread(imagePath, flatten=False))
    my_image = scipy.misc.imresize(image, size=(width,height)).reshape((1, width*height*3)).T
    my_predicted_image = predict(d["w"], d["b"], my_image)
    print("It's a ", classes[int(np.squeeze(my_predicted_image))])
    return int(np.squeeze(my_predicted_image))

In [ ]:
d = model(train_X, train_Y)
    
costs = np.squeeze(d['costs'])
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (per hundreds)')
plt.title("Learning rate =" + str(d["learning_rate"]))
plt.show()
    
results = '' 
for i in range(1,12501):
    results += (str(i) + ',' + str(predictImage(f'./catsvsdogs/test/{i}.jpg')) + '\n')
     
with open('./results.csv', 'w') as f:
    f.write(results)